In [1]:
import cv2
import os
import sys
import shutil
from datetime import datetime, timezone, timedelta

cwd = os.getcwd()
delimiter = "\\" if "\\" in cwd else "/"
repoPath = delimiter.join(cwd.split(delimiter)[:cwd.split(delimiter).index("dataImport")]) + delimiter
sys.path.append(repoPath + "dataImport/")

import vidUtils
from vidUtils import deleteVidInterval, getMP4Path, bulkDataPath
import rwWorkingTSDf
from rwWorkingTSDf import readWorkingTSDF, deleteTSDfInterval, dt_to_fnString

In [2]:
deviceDescriptor = ["abhik", "mobilepi", "abhik", "piCam-raspberryPi5-Camv3120noir", "mp4", "piVidCap"]
#deviceDescriptor = ["abhik", "bedroompi", "abhik", "piCam-raspberryPi5-Camv3120", "mp4", "piVidCap"]
#deviceDescriptor = ["abhik", "bathroompi", "abhik", "piCam-raspberryPi5-Camv3noir", "mp4", "piVidCap"]
#deviceDescriptor = ["abhik", "kitchenpi", "abhik", "piCam-raspberryPi5-Camv3120noir", "mp4", "piVidCap"]

tz = timezone(timedelta(hours=-7))
delStartTime = datetime(2025, 3, 15, 21, 53, 0, 000000, tzinfo=tz)
delEndTime = datetime(2025, 3, 15, 21, 53, 45, 000000, tzinfo=tz)
#once we have a function in vidutils we could just call it here

read in 536321 rows from 1 files, retruning 0 rows


,videoIndex,videoStartTime,videoEndTime
sampleDT,,,


In [ ]:
toDelDf = readWorkingTSDF(deviceDescriptor, None, delStartTime, delEndTime)
toDelDf

In [6]:
grouped = toDelDf.groupby("videoStartTime")
groupKeys = sorted(toDelDf['videoStartTime'].unique())
allFileNames = []
for firstTs, group in grouped:
    lastTs = group["videoEndTime"].iloc[0]
    allFileNames.append(getMP4Path(deviceDescriptor, firstTs, lastTs))
allFileNames = sorted(allFileNames)
if len(allFileNames):
    pass
    #return
allFileNames

[]

In [4]:
firstCut = grouped.get_group(groupKeys[0])['videoIndex'].iloc[0] != 0
lastRow = grouped.get_group(groupKeys[-1]).iloc[-1]
lastCut = lastRow.name != lastRow['videoEndTime']

firstGroup = grouped.get_group(groupKeys[0])
lastGroup = grouped.get_group(groupKeys[-1])
output_start_ts = groupKeys[0]
output_end_ts = lastGroup['videoEndTime'].iloc[0]

IndexError: list index out of range

In [ ]:
# get output parameters from the first video
cap = cv2.VideoCapture(allFileNames[0])
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'avc1')
tempFilePath = bulkDataPath + "_".join(deviceDescriptor) + "/temp.mp4"
cap.release()

# make an outout with the temp direcotry
out = cv2.VideoWriter(tempFilePath, fourcc, fps, (width, height))


if firstCut:
    cap = cv2.VideoCapture(allFileNames[0])
    end_frame_index = firstGroup['videoIndex'].iloc[0]
    for i in range(end_frame_index):
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
    cap.release()
    if not lastCut:
        output_end_ts = firstGroup.iloc[0].name

if lastCut:
    cap = cv2.VideoCapture(allFileNames[-1])
    start_frame_index = lastGroup['videoIndex'].iloc[-1] + 1
    cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame_index)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
    cap.release()
    if not firstCut:
        output_start_ts = lastGroup.iloc[-1].name

out.release()

newFileName = getMP4Path(deviceDescriptor, output_start_ts, output_end_ts)
newFileName

In [ ]:
[os.remove(f) for f in allFileNames]

if not firstCut and not lastCut:
    #delete the temp file
    os.remove(tempFilePath)
else:
    #move the temp file to the new file name
    shutil.move(tempFilePath, newFileName)

deleteTSDfInterval(deviceDescriptor, delStartTime, delEndTime)